In [ ]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [ ]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

# download do http para arquivo local
!wget --quiet --show-progress -O clientes.csv https://raw.githubusercontent.com/JoaoVitorDeOliveira/machine-learning/master/clientes_teste.csv
df = sc.read.csv("./clientes.csv", inferSchema=True, header=True)

# ver algumas informações sobre os tipos de dados de cada coluna
df.printSchema()

clientes.csv        100%[===================>]  41.60K  --.-KB/s    in 0.006s  
root
 |-- NOME: string (nullable = true)
 |-- SOBRENOME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- DATA_NASCIMENTO: timestamp (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- ANOS_CLIENTE: integer (nullable = true)
 |-- AVALIACAO_EMPRESA: integer (nullable = true)
 |-- PARTICIPOU_DE_CAMPANHA: integer (nullable = true)
 |-- INDICARIA_EMPRESA: integer (nullable = true)



In [ ]:
df.show()

+-------+---------+--------------------+----+-------------------+-------+---------+------------+-----------------+-------------+-----------------+
|   NOME|SOBRENOME|               EMAIL|SEXO|    DATA_NASCIMENTO|SALARIO|  EMPRESA|ANOS_CLIENTE|AVALIACAO_EMPRESA|JA_PARTICIPOU|INDICARIA_EMPRESA|
+-------+---------+--------------------+----+-------------------+-------+---------+------------+-----------------+-------------+-----------------+
|Melissa|  Almeida|                    |   F|1973-01-17 00:00:00|6077.96|MovilePay|           9|                8|            1|                1|
|   Caio|     Lima|CaioAzevedoLima@a...|   M|1951-05-24 00:00:00|5202.26|MovilePay|           5|                2|            0|                1|
|Isabela|  Correia|IsabelaRibeiroCor...|   F|1978-02-28 00:00:00|6867.53|MovilePay|           2|                8|            0|                0|
|Julieta|  Martins|JulietaDiasMartin...|   F|1957-02-27 00:00:00| 7106.5|MovilePay|           6|                3|    

# Demanda 

#### *Obs: Todos os dados usados aqui são dados fictícios*

O Grupo Movile gostaria de criar uma campanha homenageando as Empreendedoras que são clientes em todas as empresas do grupo , não existe um banco de dados com todos os dados juntos, então cada empresa ficou responsável de buscar os dados de seus cliente e agregar todos em um único CSV.

Cada empresa tem suas próprias regras e cada uma vai precisar dos dados curados e com sua regra aplicada.

1.   A primeira etapa é saber quantas Empreendedoras temos no grupo da `Movile` para mensurar a campanha e vidas impactadas.

2.   O `Ifood` não vai cobrar por 3 meses Empreendedoras que possuam `SALARIO` abaixo de 6000.00

3.   A `MovilePay` vai doar crédito para as Empreendedoras em três niveis e o controle disso será por flags em uma nova coluna chamada `NIVEL_DOACAO`: com `SALARIO` abaixo de 6000.00 a empreendora recebe o número 1, de 6000.00 até 8000.00 recebe o número 2, acima disso recebe o número 3

4. A `Sympla` precisa saber quantos % a mais Empreendedores ganham comparado com Empreendedoras para uma apresetação de conscientização

5. E a `Zoop` quer saber as 10 Empreendedoras com o maior salario para oferecerem um novo tipo de conta Black.

6. Por fim é necessário montar um dataset para ser consumido por um modelo de machine learning capaz de prever se um cliente indicaria a empresa ou não.

Por serem dados consolidados de diferentes fontes há a possibilidade de dados nulos, campos em brancos e duplicados, será necessário uma limpenza para cada CSV gerado antes de aplicar as regras de negócio.

